In [1]:
from textstat.textstat import textstat
import pandas as pd
from bs4 import BeautifulSoup
from collections import Counter

In [2]:
novels = pd.read_csv("../clean data/novel_data.csv", index_col=0)
novels.head()

,Writer Name,Novel Name,Genre,Final Word Count,Daily Average,Winner,Synopses,url,Novel Date,Excerpt
0,Nicaless,Novel: Lauren's Birthday,Genre: Young Adult,24229,807,0,\n<p></p>\n,http://nanowrimo.org/participants/nicaless/nov...,November 2015,\n<p></p>\n
1,Nicaless,Novel: A Mystery in the Kingdom of Aermon,Genre: Fantasy,50919,"1,697",1,\n<p>Hitoshi is appointed the youngest Judge a...,http://nanowrimo.org/participants/nicaless/nov...,November 2014,"\n<p>This story, funnily enough, started out a..."
2,Rachel B. Moore,Novel: Finding Fortunato,Genre: Literary,50603,"1,686",1,\n<p>Sam and Anna Gold and their newly adoptiv...,http://nanowrimo.org/participants/rachel-b-moo...,November 2015,\n<p></p>\n
3,Rachel B. Moore,Novel: The Residency,Genre: Literary,50425,"1,680",1,\n<p>It's every writer's dream - an all-expens...,http://nanowrimo.org/participants/rachel-b-moo...,November 2014,\n<p></p>\n
4,Rachel B. Moore,Novel: The Jew From Fortunato,Genre: Literary Fiction,41447,"1,381",0,\n<p>20-something Andre Levinsky is a fish out...,http://nanowrimo.org/participants/rachel-b-moo...,November 2013,\n<p></p>\n


In [3]:
soup = BeautifulSoup(novels['Synopses'][2], "html.parser")
s = soup.find_all("p")
print len(s)
mystr = ""
for i in s:
    mystr = mystr + i.renderContents() + " "


print textstat.flesch_reading_ease(mystr)
print textstat.lexicon_count(mystr)
print len(Counter(mystr.split(" ")))
mystr

4
58.62
153
109


"Sam and Anna Gold and their newly adoptive family - broken and traumatized 'uncle' Leo Berger and 'brother' Josef Neumann -\xc2\xa0 have finally left the Fohrenwald Displaced Person's Camp in Germany for their new home, the small Argentinean town of Fortunato. How do you reconstruct your life from scratch, in a new land and a new language? Having left Germany with false papers declaring their familial bond, the non-traditional family is thrust into rural Fortunato with a few pesos and the promise of work and four walls to call home. Settling into a new home isn't easy when you know what you're doing, but when you've just escaped the horrors of post-Wold War II Germany, it is an incredible challenge. Here in Fortunato they will find peace and stability. They will learn Spanish. They will find work, they will find friendship, and they will become the family that their paperwork claims they already are. "

In [4]:
test = novels['Synopses'][1:10]
test

1    \n<p>Hitoshi is appointed the youngest Judge a...
2    \n<p>Sam and Anna Gold and their newly adoptiv...
3    \n<p>It's every writer's dream - an all-expens...
4    \n<p>20-something Andre Levinsky is a fish out...
5    \n<p>Andre Levinsky is a celebrated writer in ...
6    \n<p>On his last night in the US, Nicholas Cha...
7    \n<p>Abel works in the library, cataloging don...
8                                          \n<p></p>\n
9                                          \n<p></p>\n
Name: Synopses, dtype: object

In [5]:
synopses = []
num_paragraphs = []
num_words = []
num_uniques = []
num_sentences = []
for novel in novels['Synopses']:
#for novel in test:
    mystr = ""
    soup = BeautifulSoup(novel, 'html.parser')
    syn = soup.find_all("p")
    for i in syn:
        mystr = mystr + i.renderContents() + " "
    synopses.append(mystr)
    num_words.append(textstat.lexicon_count(mystr))
    if textstat.lexicon_count(mystr) == 0:
        num_sentences.append(0)
        num_paragraphs.append(0)
        num_uniques.append(0)
    else:
        num_sentences.append(textstat.sentence_count(mystr))
        num_paragraphs.append(len(syn))
        num_uniques.append(len(Counter(mystr.split(" "))))

In [6]:
scores = []
has_synopses = []
for i in synopses:
    if i == " " or textstat.sentence_count(i) == 0:
        scores.append(0)
        has_synopses.append(0)
    else:
        scores.append(textstat.flesch_reading_ease(i))
        has_synopses.append(1)

In [7]:
standard_genres = ['Adventure','Children\'s Fiction','Erotic','Fanfiction','Fantasy','Historical','Horror/Supernatural','Literary','Mainstream','Mystery','Personal','Religious/Spiritual','Romance','Satire/Humor','Science Fiction','Thriller/Suspense','Women\'s Fiction','Young Adult']

In [19]:
set(novels['Genre'])

{'Genre:',
 'Genre: (TBC)',
 'Genre: A Tarot Focused Nano Rebellion',
 'Genre: A little bit of everything',
 'Genre: ALL OF THE ABOVE',
 'Genre: Adventure',
 'Genre: Adventure/Magic Realism',
 'Genre: Adventurous Satire',
 'Genre: All of the above',
 'Genre: All. Just all the genres.',
 'Genre: Alternative history',
 'Genre: Analysis',
 'Genre: Autobiography/Cookbook',
 'Genre: Bisexual Apocalypse/YA',
 "Genre: Can't remember",
 'Genre: Chais pas (mais \xc3\xa7a sera nawak)',
 'Genre: Chick Lit',
 "Genre: Children's",
 "Genre: Children's Fiction",
 'Genre: Choose Your Own Adventure',
 'Genre: Coming of Age (A Little Late)',
 'Genre: Creative Non-fiction',
 'Genre: Crime/Drama',
 'Genre: Cristallologie picaresque',
 'Genre: Dark Fantasy',
 'Genre: Dark Fantasy/Dieselpunk',
 'Genre: Dieselpunk Fantasy',
 'Genre: Distopian Superhero',
 'Genre: Drama/Humour',
 'Genre: Dual Perspective Piece',
 'Genre: Dystopian',
 'Genre: Editing',
 'Genre: Erotic',
 'Genre: Erotic Fiction',
 'Genre: Eroti

In [22]:
genres = []
for i in novels['Genre']:
    g = i.split(": ")
    if len(g) == 1:
        genres.append("")
    else:
        genres.append(g[1])
novels['Genre'] = genres

In [23]:
has_genre = []
standard_genre = []
for genre in novels['Genre']:
    if genre == "":
        has_genre.append(0)
    else:
        has_genre.append(1)
    if genre in standard_genres:
        standard_genre.append(1)
    else:
        standard_genre.append(0)

In [24]:
#num_paragraphs
#num_words
#num_uniques
#scores
#has_synopses
#has_genre
#standard_genre
#num_sentences
novels['has genre'] = has_genre
novels['standard genre'] = standard_genre
novels['has_synopses'] = has_synopses
novels['num words'] = num_words
novels['num uniques'] = num_uniques
novels['num sentences'] = num_sentences
novels['paragraphs'] = num_paragraphs
novels['fk score'] = scores

In [25]:
novels.head()

,Writer Name,Novel Name,Genre,Final Word Count,Daily Average,Winner,Synopses,url,Novel Date,Excerpt,has genre,standard genre,has_synopses,num words,num uniques,num sentences,paragraphs,fk score
0,Nicaless,Novel: Lauren's Birthday,Young Adult,24229,807,0,\n<p></p>\n,http://nanowrimo.org/participants/nicaless/nov...,November 2015,\n<p></p>\n,1,1,0,0,0,0,0,0.00
1,Nicaless,Novel: A Mystery in the Kingdom of Aermon,Fantasy,50919,"1,697",1,\n<p>Hitoshi is appointed the youngest Judge a...,http://nanowrimo.org/participants/nicaless/nov...,November 2014,"\n<p>This story, funnily enough, started out a...",1,1,1,44,42,3,1,65.73
2,Rachel B. Moore,Novel: Finding Fortunato,Literary,50603,"1,686",1,\n<p>Sam and Anna Gold and their newly adoptiv...,http://nanowrimo.org/participants/rachel-b-moo...,November 2015,\n<p></p>\n,1,1,1,153,109,7,4,58.62
3,Rachel B. Moore,Novel: The Residency,Literary,50425,"1,680",1,\n<p>It's every writer's dream - an all-expens...,http://nanowrimo.org/participants/rachel-b-moo...,November 2014,\n<p></p>\n,1,1,1,59,51,4,3,65.73
4,Rachel B. Moore,Novel: The Jew From Fortunato,Literary Fiction,41447,"1,381",0,\n<p>20-something Andre Levinsky is a fish out...,http://nanowrimo.org/participants/rachel-b-moo...,November 2013,\n<p></p>\n,1,0,1,124,93,4,1,56.93


In [26]:
novels.to_csv("novel_features.csv")